In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
pd.set_option("display.precision", 1)
from collections import defaultdict
import csv
from numpy import genfromtxt

In [2]:
def load_data():
    ''' called to load preprepared data for the lab '''
    item_train = genfromtxt('content_item_train.csv', delimiter=',')
    user_train = genfromtxt('content_user_train.csv', delimiter=',')
    y_train    = genfromtxt('content_y_train.csv', delimiter=',')
    with open('content_item_train_header.txt', newline='') as f:    #csv reader handles quoted strings better
        item_features = list(csv.reader(f))[0]
    with open('content_user_train_header.txt', newline='') as f:
        user_features = list(csv.reader(f))[0]
    item_vecs = genfromtxt('content_item_vecs.csv', delimiter=',')

    destination_dict = defaultdict(dict)
    count = 0
#    with open('./data/movies.csv', newline='') as csvfile:
    with open('content_destinations_list.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1  #skip header
                #print(line) print
            else:
                count += 1
                destination_id = int(line[0])
                destination_dict[destination_id]["name"] = line[1]
                destination_dict[destination_id]["category"] = line[2]


    return(item_train, user_train, y_train, item_features, user_features, item_vecs, destination_dict)

In [3]:
item_train, user_train, y_train, item_features, user_features, item_vecs, destination_dict = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 437


In [4]:
print(f"{len(item_train)}, {len(user_train)}, {len(y_train)}, {len(item_features)}, {len(user_features)}, {len(item_vecs)}, {len(destination_dict)}, ")

437, 437, 437, 9, 9, 437, 437, 


In [4]:
def split_str(ifeatures, smax):
    ''' split the feature name strings to tables fit '''
    ofeatures = []
    for s in ifeatures:
        if not ' ' in s:  # skip string that already have a space
            if len(s) > smax:
                mid = int(len(s)/2)
                s = s[:mid] + " " + s[mid:]
        ofeatures.append(s)
    return ofeatures

In [5]:
def pprint_train(x_train, features, vs, u_s, maxcount=5, user=True):
    """ Prints user_train or item_train nicely """
    if user:
        flist = [".0f", ".0f", ".1f",
                 ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f"]
    else:
        flist = [".0f", ".0f", ".1f",
                 ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f"]

    head = features[:vs]
    if vs < u_s: print("error, vector start {vs} should be greater then user start {u_s}")
    for i in range(u_s):
        head[i] = "[" + head[i] + "]"
    genres = features[vs:]
    hdr = head + genres
    disp = [split_str(hdr, 5)]
    count = 0
    for i in range(0, x_train.shape[0]):
        if count == maxcount: break
        count += 1
        disp.append([x_train[i, 0].astype(int),
                     x_train[i, 1].astype(int),
                     x_train[i, 2].astype(float),
                     *x_train[i, 3:].astype(float)
                    ])
    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=flist, numalign='center')
    return table

In [21]:
print(destination_dict)

defaultdict(<class 'dict'>, {1: {'name': 'Monumen Nasional', 'category': 'Budaya'}, 2: {'name': 'Kota Tua', 'category': 'Budaya'}, 3: {'name': 'Dunia Fantasi', 'category': 'Taman Hiburan'}, 4: {'name': 'Taman Mini Indonesia Indah (TMII)', 'category': 'Taman Hiburan'}, 5: {'name': 'Atlantis Water Adventure', 'category': 'Taman Hiburan'}, 6: {'name': 'Taman Impian Jaya Ancol', 'category': 'Taman Hiburan'}, 7: {'name': 'Kebun Binatang Ragunan', 'category': 'Cagar Alam'}, 8: {'name': 'Ocean Ecopark', 'category': 'Taman Hiburan'}, 9: {'name': 'Pelabuhan Marina', 'category': 'Bahari'}, 10: {'name': 'Pulau Tidung', 'category': 'Bahari'}, 11: {'name': 'Pulau Bidadari', 'category': 'Bahari'}, 12: {'name': 'Pulau Pari', 'category': 'Bahari'}, 13: {'name': 'Pulau Pramuka', 'category': 'Bahari'}, 14: {'name': 'Pulau Pelangi', 'category': 'Bahari'}, 15: {'name': 'Pasar Seni', 'category': 'Pusat Perbelanjaan'}, 16: {'name': 'Jembatan Kota Intan', 'category': 'Budaya'}, 17: {'name': 'Museum Fatahilla

In [ ]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

In [ ]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

In [6]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [3.36666667 3.24137931 3.36666667 3.42307692 3.36363636]


In [5]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [6]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (349, 9)
movie/item test data shape: (88, 9)


In [9]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[Use r_Id],[Rating _Count],[Rating_ Average],Bah ari,Bud aya,Cagar Alam,Pusat Perbelanjaan,Taman Hiburan,Tempat Ibadah
-1,-1,-1.8,-1.4,-1.5,-1.7,0.5,0.2,0.7
-1,-1,-1.9,0.0,-1.0,-0.7,-0.4,-1.2,-1.4
0,0,-0.4,0.7,-1.2,-0.7,1.7,1.1,-0.2
-1,1,-0.2,0.2,-1.2,-1.7,0.8,1.2,-1.4
0,0,0.3,-0.4,0.2,0.3,1.7,-0.2,1.6


In [7]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
    ### END CODE HERE ###
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
    ### END CODE HERE ###
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   38816     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   39328     ['input_2[0][0]']             
                                                                                              

In [8]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [9]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
11/11 [==============================] - 2s 5ms/step - loss: 0.0725
Epoch 2/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0286
Epoch 3/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0210
Epoch 4/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0148
Epoch 5/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0117
Epoch 6/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0123
Epoch 7/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0109
Epoch 8/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0073
Epoch 9/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0067
Epoch 10/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0059
Epoch 11/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0047
Epoch 12/30
11/11 [==============================] - 0s 4ms/step - loss: 0.0042
Epoch 13/30
11/11 [==============================

In [10]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

3/3 [==============================] - 0s 8ms/step - loss: 0.0079


0.00790192000567913

In [38]:
new_user_id = 1
new_rating_ave = 0.0
new_Bahari = 15.0
new_Budaya = 10.0
new_Cagar_Alam = 10.0
new_Pusat_Perbelanjaan = 10.0
new_Taman_Hiburan = 10.0
new_Tempat_Ibadah = 0.0
new_rating_count = 0

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave, new_Bahari, new_Budaya, new_Cagar_Alam, new_Pusat_Perbelanjaan, new_Taman_Hiburan, new_Tempat_Ibadah]])

In [29]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

In [30]:
def print_pred_movies(y_p, item, movie_dict, maxcount=10):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        destination_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     destination_dict[destination_id]['name'], destination_dict[destination_id]['category']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return disp

In [39]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, destination_dict, maxcount = 10)

14/14 [==============================] - 0s 2ms/step


[['y_p', 'movie id', 'rating ave', 'title', 'genres'],
 [3.9, 9, 2.6, 'Pelabuhan Marina', 'Bahari'],
 [3.9, 10, 2.7, 'Pulau Tidung', 'Bahari'],
 [3.9, 12, 2.8, 'Pulau Pari', 'Bahari'],
 [3.9, 209, 2.2, 'Pantai Sanglen', 'Bahari'],
 [3.9, 184, 2.4, 'Pantai Sepanjang', 'Bahari'],
 [3.9, 403, 2.0, 'Taman Barunawati', 'Taman Hiburan'],
 [3.9, 292, 3.1, 'Curug Anom', 'Cagar Alam'],
 [3.9, 365, 3.1, 'Waduk Jatibarang', 'Cagar Alam'],
 [3.9, 392, 3.3, 'Ekowisata Mangrove Wonorejo', 'Cagar Alam'],
 [3.9, 167, 3.3, 'Geoforest Watu Payung Turunan', 'Cagar Alam']]

In [40]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
